1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


**Ответ: предпочтительно использовать precision@k, т.к. это дает нам возможность оценить количественную "успешность" списка рекомендованных товаров (долю), что наиболее приближено к бизнес-метрикам.**  
**Примеры использования метрики hit_rate@k:**  
**1. Рекомендация пользователю тематических подборок фильмов (клиент - пользователь, подборка - товар)**   
**2. Рекомендация пользователю в онлайн магазине подборку из одного типа сопутсвующих товаров, например покупает нож - в рекомендациях подборка разделочных досок. В другой раз - подборка точильных камней. (клиент - пользователь, подборка - товар)**

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



**Логарифм является функцией дисконтирования. Позиции в начале списка имеют больший "вес", чем в конце. Например, пользователь редко смотрит вторую страницу списка рекомендаций, соответсвенно и позиции второй страницы имеют меньший "вес".**   
**Вместо логарифма можно использовать факториал или, например 2^(2^n).**  
**Логарифм также используется при предобработке данных, например - log-трансформация (когда один признак содержит числа разных порядков)**  

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

**Fraction of Concordance Pairs (FCP) - Насколько высока концентрация интересных товаров в начале списка рекомендаций**  
**Используется число согласных пар товаров и соответствующее число диссонирующих пар**

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8 (expected reciprocal rank)  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [10]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5

In [11]:
bought_list = [1, 3, 5, 7, 9, 11]
recommended_list = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

$precison@8 = \frac{0 + 0 + 1 + 1 + 1 + 1}{1+1+1+1+1+1+1+1} = 50\%$

In [74]:
def precision_at_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [78]:
precision_at_k(recommended_list, bought_list, k=8)

0.5

$recall@8 = \frac{0 + 0 + 1 + 1 + 1 + 1}{1+1+1+1+1+1} = 66\%$

In [14]:
def recall_at_k(recommended_list, bought_list, k=8):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [15]:
recall_at_k(recommended_list, bought_list, k=8)

0.6666666666666666

$ap_k@8 = \frac{1}{8}*(\frac{1}{2} + \frac{2}{3} + \frac{3}{5} + \frac{4}{6}) = 0,30416666666\$

In [83]:
def ap_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k): #возможно я ошибаюсь, но по-моему неверно указан диапазон, по идее нужно же идти с 1 элемента (поставил range с 0)
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1) #также здесь 
            sum_ += p_k
            
    result = sum_ / len(flags[:k]) #и тут - длина flags не полная, а только до k
    return result

In [82]:
ap_k(recommended_list, bought_list, k=8)

0.30416666666666664

In [141]:
def NDCG_k(recommended_list, bought_list, k=8):
    recommended_list = np.array(recommended_list)[:k]
    #bought_list = np.array(bought_list) 

    idealDCG_k = 1
    for i in range (1, k):
        idealDCG_k += 1/np.log(i+1)
    
    if recommended_list[0] in bought_list:
        DCG_k = 1
    else: DCG_k = 0    
    for i in range(1, k):
        if recommended_list[i] in bought_list:
            DCG_k += 1/np.log(i+1)

    NDCG = DCG_k / idealDCG_k
    return NDCG

In [142]:
NDCG_k(recommended_list, bought_list, k=8)

0.5653142737255068

In [147]:
def RR_k(recommended_list, bought_list, k=8):
    ranks=0
    for rec in recommended_list[:k]:
        for i, item in enumerate(bought_list):
            if rec == item:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [148]:
RR_k(recommended_list, bought_list, k=8)

0.095